In [79]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd

In [81]:
#xpaths
closetut='//*[@id="my-text"]/img'
searchbox='//*[@id="search-text"]'

closestmrt='//*[@id="mrt_0"]'
mrtdist='//*[@id="distance"]/div[3]/div[2]'
latlon='//*[@id="start_marker"]/div[1]/div[2]'

agreebox='//*[@id="AgreeBoxSQ"]/label'
schools='//*[@id="mCSB_2_container"]/div'

In [82]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')
path="../Dependencies/chromedriver.exe"
driver = webdriver.Chrome(executable_path=path, options=options)

C:\Users\melvc\AppData\Local\Temp/ipykernel_44304/3787452338.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


In [83]:
URAdf=pd.read_csv("../Raw Data/Past transactions (condo and apartments).csv")
URAdf.head()
properties=URAdf.loc[URAdf["Tenure"]!="Freehold"]["Project Name"].unique()
print(properties)

['NEPTUNE COURT' 'VILLA MARINA' 'PEBBLE BAY' 'WATER PLACE'
 'TANJONG RIA CONDOMINIUM' 'MANDARIN GARDENS' 'LEGENDA AT JOO CHIAT'
 'LAGUNA PARK' 'THE SHORE RESIDENCES' 'CASUARINA COVE' 'LAGOON VIEW'
 'SANCTUARY GREEN' 'SEASIDE RESIDENCES' 'RIVEREDGE' "COTE D'AZUR"
 'SILVERSEA' 'COSTA RHU' 'CAMELOT BY-THE-WATER' 'THE RED HOUSE'
 'DUNMAN VIEW' 'EAST BAY GARDENS' 'RESIDENTIAL APARTMENTS' 'ROXY SQUARE'
 'FRANKEL ESTATE' 'SEASIDE PARK' 'KATONG PARK TOWERS']


MRT scrape

In [88]:
#open website
driver.get("https://www.onemap.gov.sg/main/v2/")
element = driver.find_element(By.XPATH, closetut)
#element.click()

prop_input = driver.find_element(By.ID, "search-text")
LL=[ ]
failed=[]

columns=["Project Name","MRT", "MRTdist", "lat","lon"]
df2=pd.DataFrame(columns=columns)

for prop in properties:
    prop_input.clear()
    prop_input.send_keys(prop)
    prop_input.send_keys(Keys.RETURN)
    time.sleep(2)
    
    WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH, latlon)))
    LLnew=driver.find_element(By.XPATH,latlon).text.split(', ')
    i=1

    while LLnew==LL and i<=10:
        i+=1
        prop_input.clear()
        prop_input.send_keys(prop)
        prop_input.send_keys(Keys.RETURN)
        time.sleep(2)
        if i==10:
            failed.append(prop)
    
    LL=LLnew
    a=driver.find_element(By.XPATH,closestmrt).text
    b=driver.find_element(By.XPATH,mrtdist).text
    c =LL[0]
    d =LL[1]
    print(prop, a,b,c,d)
    df2.loc[len(df2)]=[prop,a,b,c,d]
    
df2.loc[failed]

NEPTUNE COURT MARINE TERRACE MRT STATION [ TE27 ] 853 m 1.3070735 103.9225138


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x010F37D3]
	(No symbol) [0x01088B81]
	(No symbol) [0x00F8B36D]
	(No symbol) [0x00F6E5DA]
	(No symbol) [0x00FDF0CB]
	(No symbol) [0x00FF0A56]
	(No symbol) [0x00FDB216]
	(No symbol) [0x00FB0D97]
	(No symbol) [0x00FB253D]
	GetHandleVerifier [0x0136ABF2+2510930]
	GetHandleVerifier [0x01398EC1+2700065]
	GetHandleVerifier [0x0139C86C+2714828]
	GetHandleVerifier [0x011A3480+645344]
	(No symbol) [0x01090FD2]
	(No symbol) [0x01096C68]
	(No symbol) [0x01096D4B]
	(No symbol) [0x010A0D6B]
	BaseThreadInitThunk [0x76697D69+25]
	RtlInitializeExceptionChain [0x774DBB9B+107]
	RtlClearBits [0x774DBB1F+191]


In [ ]:
#drop failled properties
df2.set_index("Project Name")
df2.drop(["RESIDENTIAL APARTMENTS"])

In [77]:
#open website
driver.get("https://www.onemap.gov.sg/main/v2/schoolquery")
element = driver.find_element(By.XPATH, agreebox)
element.click()

prop_input = driver.find_element(By.ID, "search-text")

for prop in properties:
    prop_input.clear()
    prop_input.send_keys(prop)
    prop_input.send_keys(Keys.RETURN)
    time.sleep(2)

KeyboardInterrupt: 

In [78]:
schools='//*[@id="mCSB_2_container"]/div/div[2]/div'
element = driver.find_element(By.XPATH, schools)
element.text
http://www.onemap.gov.sg/main/v2/?lat=1.30707348859508&lng=103.922513773018

''

In [ ]:
df=pd.read_csv("../Raw Data/Edgeprop/condos_attr.csv")
df = pd.concat([df, df2], axis=1)

In [ ]:

df.to_csv("../Raw Data/Edgeprop/condos_attr.csv", index=False)